In [1]:
import torch
from torch import nn
from torch.nn import init
import numpy as np
import pandas as pd
import sys
sys.path.append('../../')
import ML as ml
import glob, os

In [2]:
df = pd.read_csv('testset.csv', sep=',')
df.status[df['status']==-1]=0

/home/kong123/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [3]:
all_features = df.iloc[:,:-1]
all_features

,u,e,a_p,inc
0,0.50,0.40,5.68,3.141593
1,0.09,0.20,5.52,0.000000
2,0.40,0.35,1.58,0.174533
3,0.43,0.53,2.20,2.094395
4,0.17,0.90,5.60,1.570796
...,...,...,...,...
99995,0.04,0.86,3.86,3.141593
99996,0.08,0.40,2.10,2.443461
99997,0.24,0.60,4.31,2.792527
99998,0.30,0.43,2.56,2.967060


In [4]:
s =set(all_features['inc'])
inc = list(s)
inc.sort()
for i in range(len(inc)):
    all_features.inc[all_features['inc']==inc[i]]='%d'%i

In [5]:
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 标准化后，每个特征的均值变为0，所以可以直接用0来替换缺失值
all_features[numeric_features] = all_features[numeric_features].fillna(0)
#set(all_features['inc'])

In [6]:
all_features = pd.get_dummies(all_features, dummy_na=False)
all_features

,u,e,a_p,inc_0,inc_1,inc_10,inc_11,inc_12,inc_13,inc_14,...,inc_17,inc_18,inc_2,inc_3,inc_4,inc_5,inc_6,inc_7,inc_8,inc_9
0,1.692878,-0.327317,1.478700,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,-1.151586,-1.019805,1.355810,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.999106,-0.500439,-1.670355,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.207238,0.122800,-1.194156,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-0.596569,1.403902,1.417255,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-1.498472,1.265404,0.080827,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
99996,-1.220963,-0.327317,-1.270962,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
99997,-0.110929,0.365170,0.426455,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99998,0.305335,-0.223444,-0.917654,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [7]:
test_features = torch.tensor(all_features.values, dtype=torch.float)
test_labels = torch.tensor(df.status.values, dtype=torch.long)
batch_size = 512
testset = torch.utils.data.TensorDataset(test_features, test_labels)
test_iter = torch.utils.data.DataLoader(testset, batch_size, shuffle=True)

In [8]:
PATH = "./net.pt"
model = torch.load(PATH)

In [9]:
test_acc = ml.eva_acc(test_iter, model)
test_acc

0.9791

In [10]:
ml.dnn_results(test_iter, model) #将dnn预测的结果导出

196

In [11]:
#读取结果
path = './'
file = glob.glob(os.path.join(path, "df**.csv"))
#合并结果
dl = []
for f in file:
    dl.append(pd.read_csv(f,))
df = pd.concat(dl)
df

,u,e,a_p,predictions,labels
0,1.415369,0.746038,-0.833167,0,0
1,-1.151586,-0.465815,0.541664,1,1
2,1.415369,1.646272,-0.940696,0,0
3,0.374712,0.399795,0.341968,1,1
4,0.721598,-1.054429,0.503261,1,1
...,...,...,...,...,...
507,-0.457814,-1.296800,0.679915,1,1
508,-0.804700,-0.258069,-0.510581,0,0
509,-1.637226,1.438526,0.595428,0,0
510,1.276615,0.849911,-0.088147,1,1


In [12]:
#保存
df.to_csv('results.csv', index=False)
#删除多余的文件
for f in file:
    os.remove(f)

In [13]:
#模型评估
ml.dnn_eva(df)

Accuracy: 0.9791, Precision: 0.9877, Recall: 0.9780, F_1: 0.9828


(0.9791, 0.9876500289711116, 0.9779682966411488, 0.982785318924618)